In [ ]:
import os
import numpy as np
import open3d as o3d
from PIL import Image
from tqdm import tqdm, trange

In [ ]:
VOXEL_SIZE = 0.01
DEPTH_SCALING_FACTOR = 1000.0
DEPTH_CUTOFF = 1.0

base_dir = '/home/roger/gaussian_feature/feat_data/tabletop_test'

In [ ]:
depth_dir = os.path.join(base_dir, 'depth')
color_dir = os.path.join(base_dir, 'color')
pose_dir = os.path.join(base_dir, 'pose')
intrinsics_mat = np.loadtxt(os.path.join(base_dir, 'intrinsics.txt'))

depth_files = sorted(os.listdir(depth_dir))
color_files = sorted(os.listdir(color_dir))
pose_files = sorted(os.listdir(pose_dir))

assert len(depth_files) == len(color_files) == len(pose_files)

In [ ]:
volume = o3d.pipelines.integration.ScalableTSDFVolume(
    voxel_length=VOXEL_SIZE,
    sdf_trunc=3 * VOXEL_SIZE,
    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8
)

for idx in trange(len(depth_files)):
    pose = np.loadtxt(os.path.join(pose_dir, pose_files[idx]))

    rgb = np.array(Image.open(os.path.join(color_dir, color_files[idx])))
    rgb = np.ascontiguousarray(rgb)
    depth = np.array(Image.open(os.path.join(depth_dir, depth_files[idx]))) / DEPTH_SCALING_FACTOR
    depth[depth > DEPTH_CUTOFF] = 0.0 # remove invalid depth
    depth = np.ascontiguousarray(depth.astype(np.float32))

    assert rgb.shape[0] == depth.shape[0] and rgb.shape[1] == depth.shape[1]
    width = rgb.shape[1]
    height = rgb.shape[0]

    rgb = o3d.geometry.Image(rgb)
    depth = o3d.geometry.Image(depth)

    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgb, depth, depth_scale=1.0, depth_trunc=4.0, convert_rgb_to_intensity=False)
    intrinsic = o3d.camera.PinholeCameraIntrinsic(width=width,
                                                  height=height,
                                                  fx=intrinsics_mat[0, 0],
                                                  fy=intrinsics_mat[1, 1],
                                                  cx=intrinsics_mat[0, 2],
                                                  cy=intrinsics_mat[1, 2])
    extrinsic = np.linalg.inv(pose)
    # extrinsic = pose
    volume.integrate(rgbd, intrinsic, extrinsic)

In [ ]:
mesh = volume.extract_triangle_mesh()

In [ ]:
o3d.visualization.draw_geometries([mesh])